* Applying the Sigmoid Signal on TEC Token Holders
* Checking on and off signals

In [ ]:
import param as pm
import panel as pn
import holoviews as hv
import pandas as pd
import numpy as np
import hvplot.pandas


# For Pie Charts
from math import pi
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.palettes import Category20

In [ ]:
df = pd.read_csv('output/TEGR1.csv')

In [ ]:
df

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,last_score_timestamp,rawScore,threshold,balance_tec,tec_tokens_flag,balance_tea,tea_flag
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,2023-04-25 13:48:59.888771+00:00,28.57,15,NaN,NaN,3.0,1.0
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,2023-04-25 16:55:55.447871+00:00,27.21,15,NaN,NaN,NaN,NaN
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,2023-04-25 17:25:19.667155+00:00,28.57,15,NaN,NaN,NaN,NaN
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,2023-04-25 17:07:33.303578+00:00,23.56,15,NaN,NaN,NaN,NaN
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,2023-04-25 17:07:33.303578+00:00,23.56,15,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x9E669c0A,0x00000000,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,2023-05-09 22:47:57.682680+00:00,29.74,15,NaN,NaN,NaN,NaN
249,0xa21ca1aa,0xec026845,16,0x9E669c0A,0x00000000,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,2023-05-09 23:03:19.951021+00:00,21.07,15,NaN,NaN,NaN,NaN
250,0x634b5156,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,2023-05-09 23:03:19.951021+00:00,21.07,15,NaN,NaN,NaN,NaN
251,0x4efa29aa,0xcf3165f4,10,0x9E669c0A,0x00000000,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,2023-05-09 23:24:15.932770+00:00,18.04,15,NaN,NaN,NaN,NaN


In [ ]:
class TECQFSME(pm.Parameterized):
    boost_factor = pm.Number(1, bounds=(0,4), step=0.1, doc="Multiplicative factor to apply to boosting coefficient.")
    dataset  = pm.DataFrame(columns={'amountUSD', 'projectId'}, precedence=-1, doc="Dataset of donations. Must contain amountUSD and projectId columns.")
    matching_pool = pm.Integer(25_000, bounds=(0, 100_000), step=5_000, doc="Matching pool amount.")
    total_donations = pm.Number(0, constant=True, doc="Summation of amountUSD from donations dataset.")
    total_funding_pool = pm.Number(0, constant=True, doc="Summation of matching_pool and total_donations.")
    allocations  = pm.DataFrame(precedence=-1, doc="Percentages allocation table.")
    results  = pm.DataFrame(precedence=-1, doc="Matched and unmatched funding amounts. Allocation percentages times funding amounts.")
    
    def __init__(self, **params):
        super().__init__(**params)
        self.dataset = self.qf(self.dataset)
        self.update()
        
    @staticmethod
    def qf(df, column_name='amountUSD', new_column_name='quadratic_amount'):
        """
        Takes a specefied column as the donations column. Applies the QF algorithm to produce a new specefied column and intermediate calculation columns.
        """
        df = df.copy(deep=True)
        df[f'{column_name}_allocation'] = df[column_name] / df[column_name].sum()
        df[f'sqrt({column_name})'] = np.sqrt(df[column_name])
        df[f'sum(sqrt({column_name}))'] = df.groupby('projectId')[f'sqrt({column_name})'].transform('sum')
        df[f'sq(sum(sqrt({column_name})))'] = df[f'sum(sqrt({column_name}))'].transform(lambda x: x**2)
        df[f'{new_column_name}_allocation'] = df[f'sq(sum(sqrt({column_name})))'] / df[f'sq(sum(sqrt({column_name})))'].sum()
        df[new_column_name] = df[column_name].sum() * df[f'{new_column_name}_allocation']
        return df
    
    @staticmethod
    def signal_boost_v1(df, boost_factor, boost_column='amountUSD', new_column_name='amount_boosted'):
        """
        Given a dataset and a specefied column, applies the flag boost algorithm.
        Requires that the dataset contain 'tec_token_flag' and 'tea_flag'.
        """
        df['coefficient'] = 1 + boost_factor * (df['tec_tokens_flag'].astype(int) | df['tea_flag'].astype(int))
        df[new_column_name] = df[boost_column] * df['coefficient']
        return df

    
    @pm.depends('boost_factor', 'matching_pool', watch=True)
    def update(self):
        # Update total donations and funding pool
        with pm.edit_constant(self):
            self.total_donations = self.dataset['amountUSD'].sum()
            self.total_funding_pool = self.matching_pool + self.total_donations
        
        with pm.parameterized.batch_call_watchers(self):
            # Generate and apply the signal boosting coefficient
            self.dataset = self.signal_boost_v1(self.dataset, self.boost_factor, boost_column='amountUSD', new_column_name='amount_boosted')

            # Compute the Boosted Allocation
            self.dataset = self.qf(self.dataset, column_name='amount_boosted', new_column_name='quadratic_amount_boosted')

            # Remove the intermediate steps
            self.dataset = self.dataset[self.dataset.columns[~self.dataset.columns.str.contains('sqrt')]]

            # Create an allocations table that contains allocation percentages grouped and summed by project. 
            allocation_columns = ['projectId'] + list(self.dataset.columns[self.dataset.columns.str.contains('allocation')])
            self.allocations = self.dataset[allocation_columns].groupby('projectId').sum()

            # Generate the unmatched results table by multiplying allocation percentages by total donations
            unmatched_results = self.total_donations * self.allocations
            
            # Generate the matched results table by multiplying allocation percentages by total donations plus matching pool
            matched_results = self.total_funding_pool * self.allocations
            
            # Merge matched and unmatched results
            self.results = unmatched_results.merge(matched_results, left_index=True, right_index=True, suffixes=('_unmatched', '_matched'))
            
            # Sort results by funding amount
            self.results = self.results.sort_values('quadratic_amount_allocation_matched', ascending=False)

            # Save the boosting percentage stat
            self.results['Percentage Boost'] = 100 * ((self.results['quadratic_amount_boosted_allocation_matched'] - self.results['quadratic_amount_allocation_matched']) / self.results['quadratic_amount_allocation_matched'] + 1)

        
    @pm.depends('dataset')
    def view_expertise_signal(self):
        return self.results.rename({
            'quadratic_amount_allocation_unmatched': 'QF Unmatched', 
            'quadratic_amount_allocation_matched': 'QF Matched', 
            'quadratic_amount_boosted_allocation_matched': 'QF Matched + SME',
        }, axis=1).hvplot.bar(
            y=['QF Unmatched', 'QF Matched', 'QF Matched + SME'],
            rot=45,
            stacked=False,
            title="Adding Expertise into the QF Signal",
        ).opts(multi_level=False, legend_position='top_right')
    
    @pm.depends('dataset')
    def view_percentage_boost(self):
        return self.results.hvplot.bar(
            y='Percentage Boost', 
            color='purple', 
            ylim=(0, 180), 
            yformatter="%.0f%%", 
            yticks=list(range(0,200,20)),
            grid=True,
            height=400,
            rot=45,
            title="SME Boost Funding as % of QF Funding by Project",
        ) * hv.HLine(100)
    
    @pm.depends('dataset')
    def view_sme_signal_boost_by_donation(self):
        return self.dataset.hvplot.area(
            y='amountUSD', y2='amount_boosted', 
            logy=True, color='green', title="Logarithmic SME Signal Boost by Donation", 
            xlabel='index')
            
    
    def view(self):
        return pn.Row(self.param, pn.Column(self.view_sme_signal_boost_by_donation, self.view_percentage_boost))

In [ ]:
tec_qf_sme = TECQFSME(dataset=df.copy(deep=True).replace(np.nan, 0))

In [ ]:
tec_qf_sme.view()

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=4, name='Boost factor', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(description='Summation of a..., disabled=True, name='Total donations', value=6069.72067821)
        [4] FloatInput(description='Summation of m..., disabled=True, name='Total funding pool', value=31069.72067821)
    [1] Column
        [0] ParamMethod(method, _pane=HoloViews, defer_load=False)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

In [ ]:
dataset = tec_qf_sme.dataset

In [ ]:
dataset

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,...,tec_tokens_flag,balance_tea,tea_flag,amountUSD_allocation,quadratic_amount_allocation,quadratic_amount,amount_boosted,amount_boosted_allocation,quadratic_amount_boosted_allocation,quadratic_amount_boosted
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,2,...,0.0,3.0,1.0,0.001513,0.002959,17.961853,18.368665,0.001982,0.003491,32.355765
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1,...,0.0,0.0,0.0,0.000675,0.000185,1.124492,4.094567,0.000442,0.000169,1.563372
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1,...,0.0,0.0,0.0,0.012265,0.002593,15.737962,74.446665,0.008032,0.002711,25.130407
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1,...,0.0,0.0,0.0,0.000920,0.001863,11.309989,5.583500,0.000602,0.001916,17.760415
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1,...,0.0,0.0,0.0,0.000920,0.003139,19.051858,5.583500,0.000602,0.003364,31.177870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x9E669c0A,0x00000000,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1,...,0.0,0.0,0.0,0.000304,0.002593,15.737962,1.847803,0.000199,0.002711,25.130407
249,0xa21ca1aa,0xec026845,16,0x9E669c0A,0x00000000,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1,...,0.0,0.0,0.0,0.000304,0.001863,11.309989,1.843793,0.000199,0.001916,17.760415
250,0x634b5156,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1,...,0.0,0.0,0.0,0.000304,0.001476,8.957351,1.843793,0.000199,0.001242,11.514377
251,0x4efa29aa,0xcf3165f4,10,0x9E669c0A,0x00000000,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1,...,0.0,0.0,0.0,0.000304,0.000050,0.301167,1.843793,0.000199,0.000043,0.400228


In [ ]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', logy=False, color='green', title="SME Signal Boost by Donation", width=1000, height=500, ylim=(0,None), xlabel='index')

:Area   [index]   (amountUSD,amount_boosted)

In [ ]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', logy=True, color='green', title="Logarithmic SME Signal Boost by Donation", width=1000, height=500, xlabel='index')

:Area   [index]   (amountUSD,amount_boosted)

In [ ]:
from functools import reduce

# Compute the difference between consecutive indices for each group
dataset['diff'] = dataset.groupby('voter').apply(lambda x: x.index.to_series().diff().fillna(0)).reset_index(level=0, drop=True)

# Use the cumulative sum of these differences as a helper series
dataset['block'] = (dataset['diff'] > 1).groupby(dataset['voter']).cumsum()

# Group by 'Category' and the helper series 'block'
grouped = dataset.groupby(['voter', 'block'])

# Initialize an empty plot
plots = []

# Iterate over each group and create an hvplot visualization
for name, group in grouped:
    plot = group.hvplot.area(label=name[0], y='amountUSD', y2='amount_boosted', logy=True, title="Logarithmic SME Signal Boost by Donation by Donor", width=1000, height=500, xlabel='index')
    plots.append(plot)
    
combined_plot = reduce(lambda x, y: x * y, plots)

dataset.hvplot.step(y='amountUSD', logy=True, color='green', title="Logarithmic SME Signal Boost by Donation Colored by Donor", width=1000, height=500, xlabel='index') * combined_plot

:Overlay
   .Curve.I              :Curve   [index]   (amountUSD)
   .Area.A_0x01d3a04a    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x04e8e5aa    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x0f465406    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x109991e3    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x114b242d    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x12137de8    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x1626255e    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x2dd2036c    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x33878e07    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x364f0ad6    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x3b067af8    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x3e31155a    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x3e7f0545    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x410d86e3    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4194ce73    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4318cc44    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x434f5325.I  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x434f5325.II :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4405f427    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x468fd68b    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4b2f6d3a    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4c9f98b5    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x4f8c531d    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x5136cdfc    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x524cb61b    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x5658f4b3    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x56c246b0    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x66df5a77    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x684ce401    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x6b5918d8    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x70469299    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x751747b1    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x7ab13936    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x7d547666    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x7ee9033a    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x839395e2    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x84ad6912    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x89f04f5c    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x8b5d7121    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x8bf1c501    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x8f4ad500    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x9390fa86    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x94e9b636    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x9885a849    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x9ba96198    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0x9f55abf3    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xa2a2602b    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xa2b16c27    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xab612504    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xae45a838    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xae6ef687    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xaedf9a1f    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xb08f0ca7    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xb10dd042    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xb27daba2    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xb2d60143    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0xb5

In [ ]:
# Compute the difference between consecutive indices for each group
dataset['diff'] = dataset.groupby('projectId').apply(lambda x: x.sort_index().index.to_series().diff().fillna(0)).reset_index(level=0, drop=True)

# Use the cumulative sum of these differences as a helper series
dataset['block'] = (dataset['diff'] > 1).groupby(dataset['projectId']).cumsum()

# Group by 'Category' and the helper series 'block'
grouped = dataset.groupby(['projectId', 'block'])

from functools import reduce

# Initialize an empty plot
plots = []

# Iterate over each group and create an hvplot visualization
for name, group in dataset.sort_values('projectId').reset_index(drop=True).groupby('projectId'):
    plot = group.hvplot.area(
        label=name[0], 
        y='amountUSD', 
        y2='amount_boosted', 
        logy=False, title="SME Signal Boost by Donation by Project", width=1000, height=500, xlabel='index')
    plots.append(plot)
    
combined_plot = reduce(lambda x, y: x * y, plots)

# Display the combined plot
combined_plot


:Overlay
   .Area.A_0.I    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.II   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.III  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.IV   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.V    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.VI   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.VII  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.VIII :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.IX   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.X    :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XI   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XII  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XIII :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XIV  :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XV   :Area   [index]   (amountUSD,amount_boosted)
   .Area.A_0.XVI  :Area   [index]   (amountUSD,amount_boosted)

In [ ]:
dataset['boosted'] = dataset['amount_boosted'] - dataset['amountUSD']

boosted = dataset[dataset['boosted'] > 0]

In [ ]:
from functools import reduce

# Initialize an empty plot
plots = []

# Iterate over each group and create an hvplot visualization
for name, group in boosted.sort_values('projectId').reset_index(drop=True).groupby('projectId', as_index=False):
    plot = group.hvplot.bar(
        label=name[0], 
        y='boosted',
        x='index',
        logy=True, 
        title="Logarithmic SME Signal Boost by Donation by Project", 
        width=1000, height=500, rot=45,
        xlabel='index',
    ).opts(logy=True, ylim=(1e-1, None))
    plots.append(plot)
    
combined_plot = reduce(lambda x, y: x * y, plots)

# Display the combined plot
combined_plot


:Overlay
   .Bars.A_0.I    :Bars   [index]   (boosted)
   .Bars.A_0.II   :Bars   [index]   (boosted)
   .Bars.A_0.III  :Bars   [index]   (boosted)
   .Bars.A_0.IV   :Bars   [index]   (boosted)
   .Bars.A_0.V    :Bars   [index]   (boosted)
   .Bars.A_0.VI   :Bars   [index]   (boosted)
   .Bars.A_0.VII  :Bars   [index]   (boosted)
   .Bars.A_0.VIII :Bars   [index]   (boosted)
   .Bars.A_0.IX   :Bars   [index]   (boosted)
   .Bars.A_0.X    :Bars   [index]   (boosted)
   .Bars.A_0.XI   :Bars   [index]   (boosted)
   .Bars.A_0.XII  :Bars   [index]   (boosted)
   .Bars.A_0.XIII :Bars   [index]   (boosted)
   .Bars.A_0.XIV  :Bars   [index]   (boosted)
   .Bars.A_0.XV   :Bars   [index]   (boosted)
   .Bars.A_0.XVI  :Bars   [index]   (boosted)

In [ ]:
boosted.sort_values('projectId').rename_axis('Index').reset_index(drop=False).hvplot.bar(
        label=name[0], 
        y='boosted',
        x='Index',
        logy=True, 
        title="Logarithmic SME Signal Boost by Donation by Project", 
        width=1000, height=500, rot=90,
        xlabel='index',
        color='projectId',
        cmap='Category20',
    ).opts(logy=True, ylim=(1e-1, None))

:Bars   [Index]   (boosted,projectId)

In [ ]:
boosted.hvplot.bar(
        label=name[0], 
        y='boosted',
        x='index',
        logy=True, 
        title="Logarithmic SME Signal Boost by Donation by Project", 
        width=1000, height=500, rot=90,
        xlabel='index',
        color='projectId',
        cmap='Category20',
    ).opts(logy=True, ylim=(1e-1, None))

:Bars   [index]   (boosted,projectId)

In [ ]:
boosted.hvplot.bar(
        label=name[0], 
        y='boosted',
        x='index',
        logy=True, 
        title="Logarithmic SME Signal Boost by Donation by Donor", 
        width=1200, height=600, rot=90,
        xlabel='index',
        color='voter',
        cmap='Category20',
    ).opts(ylim=(1e-1, None))

:Bars   [index]   (boosted,voter)

In [ ]:
data = boosted.groupby('voter', sort=False)['boosted'].sum().to_frame(name='Boosted Funding')
data.reset_index().hvplot.bar(
    rot=90,
    title="Directly Boosted Funding Amounts Per Donor",
    height=740,
    width=1100,
    color='voter',
    x='voter',
    cmap='Category20',
    logy=True,
).opts(ylim=(1e-1, None))

:Bars   [voter]   (Boosted Funding)

In [ ]:
data['angle'] = data['Boosted Funding']/data['Boosted Funding'].sum() * 2*pi

# Use the Category20 colormap and cycle through it if there are more than 20 categories
colors = [Category20[20][i % 20] for i in range(len(data))]
data['color'] = colors

p = figure(height=650, title="Donor Boosting Distribution", toolbar_location=None,
           tools="hover", tooltips="Donor: @voter\n Boosted Funding: $@{Boosted Funding}{1,11}", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4, 
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='voter', source=data.reset_index())

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [ ]:
dataset['Donation Index per Project'] = dataset.groupby('projectId').cumcount() + 1

In [ ]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted',  by='projectId', x='Donation Index per Project', stacked=False, alpha=1, title="SME Signal Boost by Donation by Project", width=1000, height=500, ylim=(0,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [projectId]
   :Area   [Donation Index per Project]   (amountUSD,amount_boosted)

In [ ]:
dataset.hvplot.area(y='amountUSD', y2='amount_boosted', by='projectId', x='Donation Index per Project', stacked=False, alpha=1, logy=True, title="Logarithmic SME Signal Boost by Donation by Project", width=1000, height=500).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [projectId]
   :Area   [Donation Index per Project]   (amountUSD,amount_boosted)

In [ ]:
dataset['Donation Index per Donor'] = dataset.groupby('voter').cumcount() + 1

In [ ]:
dataset.hvplot.area(
    y='amountUSD', y2='amount_boosted',  by='voter', x='Donation Index per Donor', stacked=False, alpha=1, 
    title="SME Signal Boost by Donation by Donor",
    width=1000, height=500, ylim=(0,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [voter]
   :Area   [Donation Index per Donor]   (amountUSD,amount_boosted)

In [ ]:
dataset.sort_values('amount_boosted', ascending=False)

,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,...,quadratic_amount,amount_boosted,amount_boosted_allocation,quadratic_amount_boosted_allocation,quadratic_amount_boosted,diff,block,boosted,Donation Index per Project,Donation Index per Donor
136,0x15ed0804,0x23387567,9,0x9E669c0A,0x00000000,0xb08f0ca7,0xbbD107D7,3.000000e+17,565.651322,2,...,20.636567,1131.302644,0.122049,0.004318,40.021352,8.0,5,565.651322,6,9
142,0xf72239dc,0xdd9b885d,13,0x9E669c0A,0x00000000,0x109991e3,0x75D923fB,5.000000e+17,936.545861,1,...,67.583369,936.545861,0.101038,0.009746,90.336852,2.0,9,0.000000,11,1
54,0x259f62a8,0xdd9b885d,13,0x9E669c0A,0x6B175474,0x434f5325,0x75D923fB,4.200000e+20,419.712378,2,...,67.583369,839.424756,0.090560,0.009746,90.336852,19.0,3,419.712378,4,1
128,0x74c047af,0x23387567,9,0x9E669c0A,0x00000000,0xb08f0ca7,0xbbD107D7,2.000000e+17,379.203690,2,...,20.636567,758.407380,0.081820,0.004318,40.021352,27.0,4,379.203690,5,1
71,0x3318a9b6,0xdd9b885d,13,0x9E669c0A,0x6B175474,0xe594469f,0x75D923fB,4.210000e+20,419.729868,1,...,67.583369,419.729868,0.045282,0.009746,90.336852,16.0,4,0.000000,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,0xefdbd106,0x64a30a4b,19,0x9E669c0A,0x00000000,0xcee0b5f0,0xA26d6AEB,2.000000e+14,0.368890,1,...,17.961853,0.368890,0.000040,0.003491,32.355765,16.0,14,0.000000,16,3
241,0x4bc76015,0x8d6f0c7b,3,0x9E669c0A,0x00000000,0xcee0b5f0,0x808DB7C1,2.000000e+14,0.368890,1,...,21.078681,0.368890,0.000040,0.003976,36.850233,4.0,23,0.000000,24,4
238,0x2e14e84d,0xa9bdf738,29,0x9E669c0A,0x00000000,0xcee0b5f0,0x5041A1C1,2.000000e+14,0.368890,1,...,19.051858,0.368890,0.000040,0.003364,31.177870,22.0,24,0.000000,25,1
243,0x1f8ac179,0xf1f4942d,24,0x9E669c0A,0x00000000,0xcee0b5f0,0x4f8c531d,2.000000e+14,0.368890,1,...,8.957351,0.368890,0.000040,0.001242,11.514377,13.0,11,0.000000,12,6


In [ ]:
dataset.hvplot.area(
    y='amountUSD', y2='amount_boosted',  by='voter', x='Donation Index per Donor', stacked=False, alpha=1, 
    title="Logarithmic SME Signal Boost by Donation by Donor", logy=True,
    width=1000, height=500, ylim=(1e-1,None)).opts(legend_position='top_right')#.opts(hv.opts.Area(color=hv.Palette('Category20')))

:NdOverlay   [voter]
   :Area   [Donation Index per Donor]   (amountUSD,amount_boosted)

In [ ]:
boosted.sort_values('amount_boosted', ascending=False).head(20)[['id', 'projectId', 'applicationId', 'voter', 'grantAddress','amountUSD','coefficient','amount_boosted', 'boosted']]

,id,projectId,applicationId,voter,grantAddress,amountUSD,coefficient,amount_boosted,boosted
136,0x15ed0804,0x23387567,9,0xb08f0ca7,0xbbD107D7,565.651322,2,1131.302644,565.651322
54,0x259f62a8,0xdd9b885d,13,0x434f5325,0x75D923fB,419.712378,2,839.424756,419.712378
128,0x74c047af,0x23387567,9,0xb08f0ca7,0xbbD107D7,379.203690,2,758.407380,379.203690
38,0xaedf9bda,0xe6424ab2,22,0x839395e2,0x8110d1D0,199.679876,2,399.359752,199.679876
129,0xe3967215,0x64a30a4b,19,0xb08f0ca7,0xA26d6AEB,189.601845,2,379.203690,189.601845
130,0xcd61ec0c,0xe6424ab2,22,0xb08f0ca7,0x8110d1D0,189.601845,2,379.203690,189.601845
37,0xaf671999,0x97589cd1,7,0x839395e2,0x0035cC37,99.839938,2,199.679876,99.839938
225,0xfad3c681,0x8d6f0c7b,3,0x3e7f0545,0x808DB7C1,55.367738,2,110.735477,55.367738
45,0xf48511f9,0x64a30a4b,19,0x839395e2,0xA26d6AEB,49.919969,2,99.839938,49.919969
126,0xd0c532c8,0xec026845,16,0x7ee9033a,0x45b79C6b,38.116350,2,76.232699,38.116350


In [ ]:
data = boosted.groupby('projectId')['boosted'].sum().to_frame(name='Boosted Funding').sort_values('Boosted Funding', ascending=False)

# Extract the default color cycle from hvplot
default_colors = hv.plotting.util.process_cmap('Category20',  ncolors=len(data))

# Create a color map for the bar chart using the default color cycle
color_map = {cat: default_colors[i] for i, cat in enumerate(data.index)}


data.hvplot.bar(
    rot=90,
    title="Directly Boosted Funding Amounts Per Project",
    height=540,
    width=1100,
).opts(color='projectId', cmap=color_map, legend_position='top_right')

:Bars   [projectId]   (Boosted Funding)

In [ ]:
data['angle'] = data['Boosted Funding']/data['Boosted Funding'].sum() * 2*pi

# Use the Category20 colormap and cycle through it if there are more than 20 categories
colors = [Category20[20][i % 20] for i in range(len(data))]
data['color'] = colors

p = figure(height=650, title="Project Boosting Distribution", toolbar_location=None,
           tools="hover", tooltips="Project: @projectId\n Boosted Funding: $@{Boosted Funding}{1,11}", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4, 
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='projectId', source=data.reset_index())

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [ ]:
data = boosted.groupby('voter')['boosted'].sum().to_frame(name='Boosted Funding').sort_values('Boosted Funding', ascending=False)

# Extract the default color cycle from hvplot
default_colors = hv.plotting.util.process_cmap('Category20',  ncolors=len(data))

# Create a color map for the bar chart using the default color cycle
color_map = {cat: default_colors[i] for i, cat in enumerate(data.index)}


data.hvplot.bar(
    rot=90,
    title="Directly Boosted Funding Amounts Per Donor",
    height=740,
    width=1100,
).opts(color='voter', cmap=color_map, legend_position='top_right')

:Bars   [voter]   (Boosted Funding)

In [ ]:
data['angle'] = data['Boosted Funding']/data['Boosted Funding'].sum() * 2*pi

# Use the Category20 colormap and cycle through it if there are more than 20 categories
colors = [Category20[20][i % 20] for i in range(len(data))]
data['color'] = colors

p = figure(height=650, title="Donor Boosting Distribution", toolbar_location=None,
           tools="hover", tooltips="Donor: @voter\n Boosted Funding: $@{Boosted Funding}{1,11}", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4, 
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='voter', source=data.reset_index())

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [ ]:
data = dataset[['amountUSD', 'boosted']].sum().to_frame(name='Value').rename_axis('Category').reset_index()
data['Category'] = ['Default Donation Amounts', 'Boost Amounts']
data

,Category,Value
0,Default Donation Amounts,6069.720678
1,Boost Amounts,3199.535370


In [ ]:
data['angle'] = data['Value']/data['Value'].sum() * 2*pi

# Use the Category20 colormap and cycle through it if there are more than 20 categories
colors = [Category20[20][i % 20] for i in range(len(data))]
data['color'] = colors

p = figure(height=650, title="Donations Amounts vs. Boost Amounts", toolbar_location=None,
           tools="hover", tooltips="Category: @Category <br> Value: $@{Value}{1,11}", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4, 
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Category', source=data.reset_index())

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [ ]:
tec_qf_sme.allocations

,amountUSD_allocation,quadratic_amount_allocation,amount_boosted_allocation,quadratic_amount_boosted_allocation
projectId,,,,
0x10b3f00e,0.007163,0.001570,0.007283,0.001572
0x23387567,0.165409,0.037399,0.214523,0.047494
0x4cd41869,0.083889,0.177688,0.056623,0.127886
0x5351510d,0.008092,0.002346,0.008471,0.002443
0x64a30a4b,0.059451,0.050307,0.073838,0.059341
0x72b0d6a6,0.011525,0.004762,0.014014,0.005735
0x8d6f0c7b,0.037947,0.083346,0.043465,0.095413
0x97589cd1,0.045227,0.049264,0.046960,0.051512
0xa9bdf738,0.031684,0.078471,0.035111,0.084089


In [ ]:
tec_qf_sme.view()

Row
    [0] Column(margin=(5, 10), name='TECQFSME')
        [0] StaticText(value='<b>TECQFSME</b>')
        [1] FloatSlider(end=4, name='Boost factor', value=1)
        [2] IntSlider(end=100000, name='Matching pool', step=5000, value=25000)
        [3] FloatInput(description='Summation of a..., disabled=True, name='Total donations', value=6069.72067821)
        [4] FloatInput(description='Summation of m..., disabled=True, name='Total funding pool', value=31069.72067821)
    [1] Column
        [0] ParamMethod(method, _pane=HoloViews, defer_load=False)
        [1] ParamMethod(method, _pane=HoloViews, defer_load=False)

In [ ]:
tec_qf_sme.view_expertise_signal()

:Bars   [projectId,Variable]   (value)